In [ ]:
import pandas as pd
import numpy as np
import pickle

from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE

import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
test_file = './test.csv'

In [ ]:
train_file= './train.csv'

In [ ]:
test_df = pd.read_csv(test_file)

In [ ]:
train_df = pd.read_csv(train_file)

In [ ]:
test_df.info()

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
train_df['Id'] = train_df['Id'].astype('str')
train_df['Id'].dtype

## Обзор количественных переменных

In [ ]:
train_num_features = train_df.select_dtypes(include=['float64', 'int64'])
train_num_features.describe()

In [ ]:
train_num_features.hist(figsize=(10,10), bins=30);

In [ ]:
plt.figure(figsize = (10, 6))

sns.heatmap(train_num_features.corr(), annot=True, linewidths=.5, cmap='YlGnBu')

plt.title('Correlation matrix')
plt.show()

### Обработка пропусков

In [ ]:
train_df.isna().sum(axis=0)

In [ ]:
# Заменяем отсутствующие значения LifeSquare на средние
train_df.loc[train_df['LifeSquare'].isna(), 'LifeSquare'] = train_df['LifeSquare'].mean()

In [ ]:
train_df['Healthcare_1'].describe()

In [ ]:
# Изучаем Healthcare_1
train_df['Healthcare_1'].hist()
plt.ylabel('count')
plt.xlabel('Healthcare_1')
plt.title('Distribution of Healthcare_1')
plt.show()

In [ ]:
# Заменяем Healthcare_1 на среднее
train_df.loc[train_df['Healthcare_1'].isna(), 'Healthcare_1'] = train_df['Healthcare_1'].mean()

### Поиск и обработка выбросов

In [ ]:
# Дома без комнат
id_for_test = train_df.loc[train_df['Rooms'] <= 0]['Id']
train_df[train_df['Rooms'] <= 0]

In [ ]:
# Меняем на mode()
train_df.loc[train_df['Rooms'] <= 0, 'Rooms'] = train_df['Rooms'].mode()[0]

In [ ]:
# Прверяем
train_df.loc[train_df['Id'].isin(id_for_test)]

In [ ]:
# Жилая площадь больше общей
id_for_test = train_df.loc[train_df['Square'] < train_df['LifeSquare']]['Id']
train_df.loc[train_df['Square'] < train_df['LifeSquare']]

In [ ]:
# Меняем местами столбцы, где жилая площадь больше общей
sq = train_df.loc[train_df['Square'] < train_df['LifeSquare'], ['LifeSquare', 'Square']]
sq.rename(columns={'LifeSquare': 'Square', 'Square': 'LifeSquare'}, inplace=True)
train_df.loc[train_df['Square'] < train_df['LifeSquare'], ['Square', 'LifeSquare']] = sq

In [ ]:
# Проверяем
train_df.loc[train_df['Id'].isin(id_for_test)]

In [ ]:
# Общая площадь меньше кухни
id_for_test = train_df.loc[train_df['Square'] < train_df['KitchenSquare']]['Id']
train_df[train_df['Square'] < train_df['KitchenSquare']]

In [ ]:
# Заменим площадь кухни на остаток от общей-жилой
train_df.loc[train_df['Square'] < train_df['KitchenSquare'], 'KitchenSquare'] = train_df.loc[train_df['Square'] < train_df['KitchenSquare'], 'Square']-train_df.loc[train_df['Square'] < train_df['KitchenSquare'], 'LifeSquare']

In [ ]:
# Проверяем
train_df.loc[train_df['Id'].isin(id_for_test)]

In [ ]:
# Ищем дома с <=0 этажей
id_for_test = train_df[train_df['HouseFloor'] <= 0]['Id']
train_df[train_df['HouseFloor'] <= 0]

In [ ]:
# Предположим, что квартиры на последних этажах
train_df.loc[train_df['HouseFloor'] <= 0, 'HouseFloor'] = train_df.loc[train_df['HouseFloor'] <= 0, 'Floor']

In [ ]:
# Этаж квартиры больше этажей дома
id_for_test = train_df.loc[train_df['Floor'] > train_df['HouseFloor']]['Id']
train_df.loc[train_df['Floor'] > train_df['HouseFloor']]

In [ ]:
# Приведем HouseFloor к int64 и поменяем местами этажи квартир и этажность домов
train_df['HouseFloor'] = train_df['HouseFloor'].astype('int64')
swap = train_df.loc[train_df['HouseFloor'] < train_df['Floor'], ['Floor', 'HouseFloor']]
swap.rename(columns={'HouseFloor': 'Floor', 'Floor': 'HouseFloor'}, inplace=True)
train_df.loc[train_df['HouseFloor'] < train_df['Floor'], ['Floor', 'HouseFloor']] = swap

In [ ]:
# Проверяем
train_df.loc[train_df['Id'].isin(id_for_test)]

In [ ]:
# Проверка года постройки
train_df['HouseYear'].value_counts()

In [ ]:
train_df.loc[(train_df['HouseYear'] < 1900) | (train_df['HouseYear'] > 2020)]

In [ ]:
train_df.loc[train_df['Id'] == 10814, 'HouseYear'] = 2005
train_df.loc[train_df['Id'] == 11607, 'HouseYear'] = 1968

In [ ]:
train_df

### Подготовка 

In [ ]:
X_train = train_df.drop(['Id','Price'], axis=1)

In [ ]:
X_train = pd.get_dummies(X_train)
X_train.info()

In [ ]:
y_train = train_df['Price']
y_train

### Обработка test_df

In [ ]:
test_df.info()

In [ ]:
test_df.loc[test_df['LifeSquare'].isna(), 'LifeSquare'] = train_df['LifeSquare'].mean()

In [ ]:
test_df.loc[test_df['Healthcare_1'].isna(), 'Healthcare_1'] = train_df['Healthcare_1'].mean()

In [ ]:
test_ids = test_df['Id']

In [ ]:
test_df.drop('Id', axis=1, inplace=True)

In [ ]:
test_df

In [ ]:
X_test = pd.get_dummies(test_df)
X_test.info()

### Стандартизация признаков

In [ ]:
scaler = StandardScaler()

In [ ]:
features_for_scale = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
features_for_scale

In [ ]:
# Стандартизируем train
X_train_scaled = scaler.fit_transform(X_train[features_for_scale])
X_train_scaled

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features_for_scale)

In [ ]:
X_train[features_for_scale] = X_train_scaled
X_train

In [ ]:
# Стандартизируем test
X_test_scaled = scaler.transform(X_test[features_for_scale])
X_test_scaled

In [ ]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns=features_for_scale)
X_test_scaled

In [ ]:
X_test[features_for_scale] = X_test_scaled
X_test

## Построение модели

In [ ]:
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 3)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 3)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 3)))

In [ ]:
parameters = [{'n_estimators': [100, 500, 1000],
               'max_depth': np.arange(5,12)}]

In [ ]:
gs = GridSearchCV(estimator=RandomForestRegressor,
                  param_grid=parameters,
                  scoring='r2',
                  cv=5,
                  n_jobs=-1)

In [ ]:
rf = RandomForestRegressor(n_estimators=2000, max_depth=18, random_state=100)

In [ ]:
gs.best_params_

In [ ]:
rf.fit(X_train.drop('Healthcare_1', axis=1), y_train)

In [ ]:
y_train_pred = rf.predict(X_train.drop('Healthcare_1', axis=1))
evaluate_preds(y_train.values.flatten(), y_train_pred.flatten())

In [ ]:
y_test_pred = rf.predict(X_test.drop('Healthcare_1', axis=1))
submission = pd.DataFrame()
submission['Id'] = test_ids
submission['Price'] = y_test_pred
submission

In [ ]:
submission.to_csv('./submission_cluster.csv', index=False)

In [ ]:
with open('./minmax.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
with open('./rfr_model_minmax_16_210.pkl', 'wb') as file:
    pickle.dump(rf, file)